In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from SALib.analyze import sobol
from SALib.sample import saltelli
from matplotlib import offsetbox
from sklearn import manifold
from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
def plot_embedding(X, y, title=None):
    x_min, x_max = np.min(X, 0), np.max(X, 0)
    X = (X - x_min) / (x_max - x_min)

    plt.figure()
    for i in range(X.shape[0]):
        plt.text(X[i, 0], X[i, 1], str(y[i]),
                 color=plt.cm.tab20((y[i]+0.5) / 20.),
                 fontdict={'weight': 'bold', 'size': 6})

    if hasattr(offsetbox, 'AnnotationBbox'):
        # only print thumbnails with matplotlib > 1.0
        shown_images = np.array([[1., 1.]])  # just something big
        for i in range(X.shape[0]):
            dist = np.sum((X[i] - shown_images) ** 2, 1)
            if np.min(dist) < 4e-3:
                # don't show points that are too close
                continue
            shown_images = np.r_[shown_images, [X[i]]]
            # imagebox = offsetbox.AnnotationBbox(
            #     offsetbox.OffsetImage(digits.images[i], cmap=plt.cm.gray_r),
            #     X[i])
            # ax.add_artist(imagebox)
    plt.xticks([]), plt.yticks([])
    if title is not None:
        plt.title(title)

In [ ]:
def calculate_fit(label_group):
    label = label_group[0]
    s = 0.
    for l in label_group[1:]:
        if l == label:
            s += 0.25
    return s


def calculate_nn_metric(X, y, neighs) -> float:
    nbrs = NearestNeighbors(n_neighbors=neighs).fit(X)
    _, indices = nbrs.kneighbors(X)
    return sum(map(lambda ind: calculate_fit(y[ind]), indices))


def perform_embedding(X, y, perplexity, early_exaggeration, learning_rate, n_iter, init, random_state, angle) -> float:
    tsne = manifold.TSNE(n_components=2, metric='cosine', perplexity=perplexity, early_exaggeration=early_exaggeration, learning_rate=learning_rate, n_iter=int(n_iter), init=('random' if init < 0.5 else 'pca'), random_state=int(random_state), angle=angle)
    return tsne.fit_transform(X)

In [ ]:
problem = {
    'num_vars': 7,
    'names': ['perplexity', 'early_exaggeration', 'learning_rate', 'n_iter', 'init', 'random_state', 'angle'],
    'bounds': [
        [5.0, 50.0],
        [1.0, 100.0],
        [10.0, 1000.0],
        [250, 2500],
        [0.0, 1.0],
        [0.0, 1000.0],
        [0.2, 0.8]
    ]
}

In [ ]:
data_text, labels = fetch_20newsgroups(remove=('headers', 'footers', 'quotes'), return_X_y=True)

In [ ]:
n_features = 60000
tfidf_vectorizer = TfidfVectorizer(max_features=n_features,
                                   stop_words='english')

data = tfidf_vectorizer.fit_transform(data_text)

data.shape


In [ ]:
X = TruncatedSVD(n_components=50).fit_transform(data)

y = labels

In [ ]:
N = 20
samples = saltelli.sample(problem, N)
np.savez('samples_tng.npz', samples)

In [ ]:
import multiprocessing
from multiprocessing import Pool, Value
import time

In [ ]:
def embedding(sample):
    global counter
    with counter.get_lock():
        counter.value += 1
        step = counter.value
    print('Starting to calculate ' + str(step) + ' of ' + str(len(samples)) + ' (sample ' + str(["{0:0.2f}".format(i) for i in sample]) + ')')
    t = time.process_time()
    embed = perform_embedding(X, y, *sample)
    elapsed_time = time.process_time() - t
    print('Calculation complete for ' + str(step) + ' of ' + str(len(samples)) + ' in ' + str(elapsed_time) + ' s.')
    return [embed, elapsed_time]

In [ ]:
counter = Value('i', 0)

with Pool(processes=multiprocessing.cpu_count()) as pool:
    embeddings_with_times = pool.map(embedding, samples)
    
np.savez('embeds_with_time.npz', np.array(embeddings_with_times))

In [ ]:
def get_metric(args):
    embedding, neighs = args
    global counter
    with counter.get_lock():
        counter.value += 1
        step = counter.value
    print('Starting to calculate ' + str(step) + ' of ' + str(len(embeddings)))
    t = time.process_time()
    metric = calculate_nn_metric(embedding, y, neighs)
    elapsed_time = time.process_time() - t
    print('Calculation complete for ' + str(step) + ' of ' + str(len(embeddings)) + ' in ' + str(elapsed_time) + ' s.')
    return metric

In [ ]:

neighs_nums = [5, 10, 20]
Ys = []
for neighs in neighs_nums:
    counter = Value('i', 0)
    print("############# Calculation of metrics for {} neighs ###################".format(neighs))
    with Pool(processes=multiprocessing.cpu_count()) as pool:
        Ys.append(pool.map(get_metric, [(embedding, neighs) for embedding in embeddings]))

